# DATA APP

In [2]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

from app_funksjoner import kjør_app
kjør_app()

ToggleButtons(button_style='info', description='Analyser:', index=2, options=('Temperaturdata', 'Klimagassutsl…

Output()